In [2]:
# Imports
import os
import json
import asyncio
import requests
from typing import cast, List, Tuple
from dotenv import load_dotenv

# Agent Framwork
from agents import Agent, Runner, OpenAIChatCompletionsModel, AsyncOpenAI, set_tracing_disabled
from opentelemetry import trace as otel_trace
from phoenix.otel import register

# Markdown output display
from IPython.display import Markdown, display

In [3]:
# Cast to String
def env_to_str(env: str) -> str:
    return cast(str, os.getenv(env))

In [4]:
# Environment Variables

# Load Environment Variables
load_dotenv(override=True)

# Ollama Environment Variables
ollama_api_key = env_to_str('OLLAMA_API_KEY')
ollama_base_url = env_to_str('OLLAMA_BASE_URL')

# Gemini Environment Variables
gemini_api_key = env_to_str("GEMINI_API_KEY")
gemini_base_url = env_to_str("GEMINI_BASE_URL")

# Phoenix Collector Variables
phoenix_collector_endpoint = env_to_str("PHOENIX_COLLECTOR_ENDPOINT") or "http://localhost:6006/v1/traces"

# Maileroo Environment Variables
maileroo_api_key = env_to_str("MAILEROO_API_KEY")
maileroo_base_url = env_to_str("MAILEROO_BASE_URL")
maileroo_template_url = env_to_str("MAILEROO_TEMPLATE_URL")

In [5]:
# Available AI Models

# Ollama General Models
model_mistral = env_to_str("MODEL_MISTRAL")
model_llama = env_to_str('MODEL_LLAMA3')
model_mistral_nemo = env_to_str("MODEL_MISTRAL_NEMO")
model_gemma = env_to_str("MODEL_GEMMA3")

# Ollama Resoning Models 
model_phi = env_to_str("MODEL_PHI3.5")
model_qwen = env_to_str("MODEL_QWEN3")
model_deepseek = env_to_str("MODEL_DEEPSEEK_R1")

# Gemini Models
model_gemini_flash = env_to_str("MODEL_GEMINI_FLASH")

In [6]:
# Attribute Access Class
class AttributeAccess:
    
    # Constructor
    def __init__(self, dictionary: dict):
        
        # Attribute Access
        for key, value in dictionary.items():
            if isinstance(value, dict):
                value = AttributeAccess(value)
            
            # Dot Notation Setter
            setattr(self, key, value)

    # Bracket Notation Getter
    def __getitem__(self, key):
        return getattr(self, key)
    
    # Bracket Notation Setter
    def __setitem__(self, key, value):
        setattr(self, key, value)
        
	# Convert Back to Python dict
    def to_dict(self) -> dict:
        result_dict = {}

        for key, value in self.__dict__.items():
            if isinstance(value, AttributeAccess):
                result_dict[key] = value.to_dict()
            else:
                result_dict[key] = value
        return result_dict

In [7]:
# Configurations

# Set Current Model
selected_model = model_llama

# Multi Model Flag
multi_model = False

# Check the need of multiple models, 
# if not, select current ai model.
if multi_model and selected_model != "":
    print("This is a multi model setup. Leave current model as empty string: ''.")
elif multi_model and selected_model == "":
    print("This is a multi model setup. ")
elif not multi_model and selected_model == "":
    print("This is a single model setup. Please set a current model.")
elif not multi_model and selected_model != "":
    print(f"This is a single model setup. The current model is: {model_llama}")

This is a single model setup. The current model is: llama3.1:8b


In [8]:
# Configure Phoenix Tracer

set_tracing_disabled(True)

if selected_model != "":
    phoenix_project_name = f"{selected_model}_phoenix"
else:
    phoenix_project_name = "multi_model_phoenix"

try:
    tracer_provider = register(
    project_name=phoenix_project_name,
    auto_instrument=True,
    endpoint=phoenix_collector_endpoint,
    set_global_tracer_provider=True
)
    print("Phoenix tracer registered successfully")
except Exception as e:
    print(f"Phoenix registration failed: {e}")

DependencyConflict: requested: "openai-agents >= 0.1.0" but found: "openai-agents 0.0.17"
DependencyConflict: requested: "google-genai" but found: "None"


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: llama3.1:8b_phoenix
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: http://localhost:6006/v1/traces
|  Transport: HTTP + protobuf
|  Transport Headers: {}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.

Phoenix tracer registered successfully


In [9]:
# Llama3 Model
llama = OpenAIChatCompletionsModel(
    model=selected_model,
    openai_client=AsyncOpenAI(
        api_key=ollama_api_key, 
        base_url=ollama_base_url
    )
)

In [10]:
# Agent Workflow Instructions
data_a = {
    "name":"Professional Sales Agent",
    "instructions":"You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails.",
    "model":llama
}
agent_a = AttributeAccess(data_a)

data_b = {
    "name":"Engaging Sales Agent",
    "instructions":"You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response.",
    "model":llama
}
agent_b = AttributeAccess(data_b)

data_c = {
    "name":"Busy Sales Agent",
    "instructions":"You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails.",
    "model":llama
}
agent_c = AttributeAccess(data_c)

data_d = {
    "name":"sales_picker",
    "instructions":"You are an expert customer focused on finding the most effective cold sales email. \
**Carefully and thoroughly evaluate each provided email option, comparing them against one another on their merits.** \
Consider which email is most compelling, clear, concise, and persuasive in its offer for SOC2 compliance. \
Based on this comprehensive evaluation, select the single email you are most likely to respond to. \
Reply with only the content of the selected best email, without any additional text or explanation.",
    "model":llama
}
agent_d = AttributeAccess(data_d)

In [11]:
# Agents

sales_agent_a = Agent(
    name=agent_a.name,
    instructions=agent_a.instructions,
    model=agent_a.model
)

sales_agent_b = Agent(
    name=agent_b.name,
    instructions=agent_b.instructions,
    model=agent_b.model
)

sales_agent_c = Agent(
    name=agent_c.name,
    instructions=agent_c.instructions,
    model=agent_c.model
)

sales_picker = Agent(
    name=agent_d.name,
    instructions=agent_d.instructions,
    model=agent_d.model
)

In [12]:
# Agentic Workflow

class AgenticWorkflow:
    # Constructor
    def __init__(
            self, 
            tracer_name: str, 
            model_name: str, 
            generation_agents: List[Agent], 
            evaluator_agent: Agent):
        self.tracer_name = tracer_name
        self.model_name = model_name
        self.generation_agents = generation_agents
        self.evaluator_agent = evaluator_agent

        # Arize Phoenix 
        # OpenTelemetry Tracer
        self.tracer = otel_trace.get_tracer(__name__)

    # Run Generation
    async def _run_generation(self, prompt: str) -> List[str]:
        with self.tracer.start_as_current_span(f"{self.tracer_name} - Generation") as span:
            span.set_attribute("phase", "generation")
            span.set_attribute("input.prompt", prompt)
            span.set_attribute("model.used", self.model_name)

            # Check for generation agents provided
            if not self.generation_agents:
                print(f"Warning: No generation agents provided for '{self.tracer_name}'.")
                return []
            
            # Run all generation agents
            agent_run_results = await asyncio.gather(
                *[Runner.run(agent, prompt) for agent in self.generation_agents]
            )

            # Extract Final Outputs
            generated_outputs = [result.final_output for result in agent_run_results]

            if generated_outputs:
                total_length = sum(len(output) for output in generated_outputs)
                span.set_attribute("generation.count", len(generated_outputs))
                span.set_attribute("generation.total_length", total_length)
                span.set_attribute("generation.first_preview", generated_outputs[0][:100] if generated_outputs[0] else "")

            return generated_outputs
    
    # Run Evaluation
    async def _run_evaluation(self, generated_content: List[str]) -> str:
            with self.tracer.start_as_current_span(f"{self.tracer_name} - Evaluation") as span:
                span.set_attribute("phase", "evaluation")
                span.set_attribute("evaluator.agent_name", self.evaluator_agent.name)

                # Combine generated content into a string
                evaluator_input = "\n\n--- Option ---\n\n".join(generated_content)

                span.set_attribute("evaluator.input_preview", evaluator_input[:500] if evaluator_input else "")

            # Run Evaluator Agent
            eval_result = await Runner.run(self.evaluator_agent, evaluator_input)
            final_output = eval_result.final_output

            span.set_attribute("evaluation.output_length", len(final_output))
            span.set_attribute("evaluation.output_preview", final_output[:100] if final_output else "")

            return final_output
    
    # Execute Workflow
    async def execute_workflow(self, initial_prompt: str) -> Tuple[List[str], str]:
        with self.tracer.start_as_current_span(self.tracer_name) as main_span:
            main_span.set_attribute("workflow.type", "generic_generation_evaluation")
            main_span.set_attribute("workflow.initial_prompt", initial_prompt)

            # Execute Generation
            generated_items = await self._run_generation(initial_prompt)

            # Execute Evaluation
            evaluated_item = await self._run_evaluation(generated_items)

            print(f"\n--- All Generated Items for '{self.tracer_name}' ---")
            if generated_items:
                for i, item in enumerate(generated_items):
                    display(Markdown(f"## Generated Item {i+1}:\n\n{item}\n\n---"))
            else:
                print("No items were generated.")

            print(f"\n--- Best Evaluated Item for '{self.tracer_name}' ---")
            if evaluated_item:
                display(Markdown(evaluated_item))
            else:
                print("No item was evaluated/picked.")


            # Final Summary Attributes
            main_span.set_attribute("final.generated_count", len(generated_items))
            main_span.set_attribute("final.evaluated_item_length", len(evaluated_item))
            main_span.set_attribute("final.evaluated_item_preview", evaluated_item[:100] if evaluated_item else "")

        return generated_items, evaluated_item

In [13]:
# Workflow Variables
tracer_name = "Parallel Cold Emails"
model_name = model_llama
generation_agents = [sales_agent_c, sales_agent_a, sales_agent_b]
evaluator_agent = sales_picker

# Init Workflow
sales_email = AgenticWorkflow(
    tracer_name, 
    model_name, 
    generation_agents, 
    evaluator_agent
)

# Run Workflow
_, best_email = await sales_email.execute_workflow("Write a cold sales email.")

Setting attribute on ended span.
Setting attribute on ended span.



--- All Generated Items for 'Parallel Cold Emails' ---


## Generated Item 1:

Here is a cold sales email:

Subject: Ensure Your SOC 2 Compliance with Ease

Hi [First Name],

As a business owner or executive, ensuring the security and confidentiality of your customers' data is a top priority. But with increasing regulations and ever-evolving threats, maintaining SOC 2 compliance can be overwhelming.

That's where ComplAI comes in - our AI-powered SaaS platform streamlines SOC 2 reporting and audits, freeing up valuable time for you to focus on growth. With automated documentation generation, risk assessments, and analytics at your fingertips, achieving and maintaining compliance has never been easier.

Schedule a 15-minute consultation with one of our experts to learn how ComplAI can support your organization's security posture.

Best,
[Alice Smith]
ComplAI Sales Agent

---

## Generated Item 2:

Subject: Ensure the Security of Your Business with ComplAI

Dear [Recipient],

As the Chief Compliance Officer at [Company], I am writing to bring to your attention an essential matter that may be impacting your organization's operational efficiency and risk management.

Ensuring SOC 2 compliance has become a critical requirement for businesses to maintain credibility, trust, and integrity with stakeholders. However, navigating the intricacies of this process can be challenging and time-consuming due to the extensive documentation and reporting required.

At ComplAI, we are a leading provider of AI-powered SaaS tools designed to streamline and simplify the SOC 2 compliance preparation process. Our innovative technology automates audit readiness by:

  * Identifying controls and policies
  * Conducting internal control assessments
  * Documenting audit trails

Our purpose-built solution ensures that you can focus on what matters most: running your business with confidence.

To learn more about ComplAI and set up a personalized demo, please click this link [insert Calendly/Cal.com meeting scheduling tool URL]. During our session, we will walk through the various features of our platform and provide valuable insights on simplifying SOC 2 compliance for your organization.

Don't let manual processes hold you back from achieving excellence in security. Let me know if you're interested or have any questions by replying to this email. I'd be happy to assist you.

Best regards,

[Your Name]

ComplAI Compliance Specialist

P.S.: As a matter of practice, we recommend preparing for SOC 2 at least one quarter ahead of schedule to ensure sufficient time for implementation, review, and iteration.

If you want me to make any changes please let me know! Also, note that it is crucial to personalize the email with correct names, titles, and company information, to maintain professionalism.

---

## Generated Item 3:

Here's one:

Subject: Your Audit Anxiety is Our Bane!

Dear [Name],

Are you feeling like your audit prep game is on the line? Like, what if they ask about our controls on data access?!?!

Chill. Just kidding (kind of). But seriously, we've been here too.

As someone responsible for keeping their organization compliant with SOC 2 standards, you know how stressful preparing for audits can be. That's where ComplAI comes in - our AI-powered SOC 2 compliance prep tool is like having your own personal audit ninja sidekick!

With ComplAI, you'll get:

✨ Customizable policies and procedures
✨ Automated tracking and monitoring
✨ Real-time risk management

So why let the audit stress take over? Join our happy tribe of clients who rave about how easy we make compliance prep. 

Take the first step towards calm audit nerves: click here [insert link] to schedule a demo with us.

Cheers, 
[Your Name]
ComplAI

P.S. No more staring at dusty audit manuals all night (your eyes thank you).

(Note: The tone is lighthearted and engaging, with an attempt to poke fun at the otherwise dry topic of compliance prep. Of course, the aim is not to come across as unprofessional but to break the ice and grab the reader's attention.)

---


--- Best Evaluated Item for 'Parallel Cold Emails' ---


ComplAI 
Ensure Your SOC 2 Compliance with Ease

Hi [First Name],

As a business owner or executive, ensuring the security and confidentiality of your customers' data is a top priority. But with increasing regulations and ever-evolving threats, maintaining SOC 2 compliance can be overwhelming.

That's where ComplAI comes in - our AI-powered SaaS platform streamlines SOC 2 reporting and audits, freeing up valuable time for you to focus on growth. With automated documentation generation, risk assessments, and analytics at your fingertips, achieving and maintaining compliance has never been easier.

Schedule a 15-minute consultation with one of our experts to learn how ComplAI can support your organization's security posture.

Best,
[Alice Smith]
ComplAI Sales Agent

In [18]:
# Maileroo
headers = {"X-API-Key": maileroo_api_key}

payload = {
    'from': 'lusanco_test@678cd71231d0ded9.maileroo.org',
    'to': 'lasc1026@gmail.com',
    'subject': 'Test Email',
    'plain': best_email,
}

response = requests.request("POST", maileroo_base_url, headers=headers, data=payload)

print(response.text)


{"data":{"reference_id":"3d0903feba4b1200480816df"},"message":"The email has been queued for delivery.","success":true}
